In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load the data
df = pd.read_json('/kaggle/input/pii-detection-removal-from-educational-data/train.json')
df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop(['full_text', 'document'], axis=1)
df.head()

In [ ]:
df.shape

In [76]:
df.isnull().sum()

tokens                 0
trailing_whitespace    0
labels                 0
dtype: int64

In [77]:
df.head()

,tokens,trailing_whitespace,labels
0,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [ ]:
import numpy as np
row = df.iloc[0]  # Get the first row
tokens = row['tokens']
trailing_whitespace = row['trailing_whitespace']
labels = row['labels']


print('tokens:', tokens)
print('trailing_whitespace:', trailing_whitespace)
print('labels:', labels)


In [ ]:
import numpy as np

# Flatten the list in the column
flattened_values = np.concatenate(df['labels'].values)

# Convert to a pandas Series
flattened_series = pd.Series(flattened_values)

# Find unique values
unique_values = flattened_series.unique()

print(unique_values)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df is the DataFrame you provided
# Remove unwanted columns
df = df[['tokens', 'trailing_whitespace', 'labels']]

# Flatten the 'labels' column
flattened_labels = [label for labels_list in df['labels'] for label in labels_list]

# Check the distribution of labels
plt.figure(figsize=(10, 6))
sns.countplot(x=flattened_labels)
plt.title("Distribution of Labels")
plt.show()

# Visualize the count of tokens in each document
df['tokens_count'] = df['tokens'].apply(len)
plt.figure(figsize=(10, 6))
sns.histplot(df['tokens_count'], bins=20, kde=True)
plt.title("Distribution of Tokens Count in Documents")
plt.xlabel("Tokens Count")
plt.ylabel("Frequency")
plt.show()

# Visualize the count of trailing whitespaces
df['whitespace_count'] = df['trailing_whitespace'].apply(lambda x: sum(x))
plt.figure(figsize=(10, 6))
sns.histplot(df['whitespace_count'], bins=20, kde=True)
plt.title("Distribution of Trailing Whitespace Count")
plt.xlabel("Whitespace Count")
plt.ylabel("Frequency")
plt.show()

# Visualize the relation between tokens count and whitespace count
plt.figure(figsize=(10, 6))
sns.scatterplot(x='tokens_count', y='whitespace_count', data=df)
plt.title("Relation between Tokens Count and Whitespace Count")
plt.xlabel("Tokens Count")
plt.ylabel("Whitespace Count")
plt.show()


In [ ]:
# import pandas as pd
# import numpy as np

# flattened_labels = np.concatenate(df['labels'].values)

# # Convert to a pandas Series
# flattened_labels_series = pd.Series(flattened_labels)

# # Find indices where label is not equal to 'O'
# non_o_indices = flattened_labels_series[flattened_labels_series != 'O'].index

# # Update 'tokens' and 'trailing_whitespace' columns based on non-'O' indices
# df['tokens'] = df['tokens'].apply(lambda x: [token for i, token in enumerate(x) if i not in non_o_indices])
# df['trailing_whitespace'] = df['trailing_whitespace'].apply(lambda x: [whitespace for i, whitespace in enumerate(x) if i not in non_o_indices])
# df['labels'] = df['labels'].apply(lambda x: [label for i, label in enumerate(x) if i not in non_o_indices])

# # Display the modified DataFrame
# print(df)


In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# List of unique labels to keep
valid_labels = ['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-EMAIL',
                'B-ID_NUM', 'I-URL_PERSONAL', 'B-USERNAME', 'B-PHONE_NUM', 'I-PHONE_NUM',
                'B-STREET_ADDRESS', 'I-STREET_ADDRESS', 'I-ID_NUM']

# Function to filter and adjust tokens and trailing_whitespace
def filter_tokens(row):
    filtered_tokens = [token for i, token in enumerate(row['tokens']) if row['labels'][i] in valid_labels]
    filtered_whitespace = [whitespace for i, whitespace in enumerate(row['trailing_whitespace']) if row['labels'][i] in valid_labels]
    return pd.Series({'tokens': filtered_tokens, 'trailing_whitespace': filtered_whitespace})

# Apply the function to each row
df[['tokens', 'trailing_whitespace']] = df.apply(filter_tokens, axis=1)

# Remove other classes from the 'labels' column
df['labels'] = df['labels'].apply(lambda labels: [label if label in valid_labels else 'O' for label in labels])

# Display the modified DataFrame
print(df)


In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

# Assuming df is your DataFrame

# Initialize tqdm for progress bar
progress_bar = tqdm(total=len(df), desc="Processing Rows", position=0)

# Track the total number of non-'O' values
total_non_o_values = 0

# Update 'tokens', 'trailing_whitespace', and 'labels' columns based on conditions
for idx, row in df.iterrows():
    tokens = row['tokens']
    labels = row['labels']

    # Identify indices where label is not equal to 'O'
    non_o_indices = [i for i, label in enumerate(labels) if label != 'O']

    # Update total_non_o_values
    total_non_o_values += len(non_o_indices)

    # Remove tokens, trailing_whitespace, and labels at non-'O' indices
    for non_o_idx in non_o_indices:
        if non_o_idx < len(tokens):
            tokens.pop(non_o_idx)
            row['trailing_whitespace'].pop(non_o_idx)
            labels.pop(non_o_idx)

    # Update tqdm
    progress_bar.update(1)

# Close tqdm
progress_bar.close()

# Display the modified DataFrame
print(f"Total Non-'O' Values: {total_non_o_values}")
df


In [ ]:
df.head()

In [78]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(data['tokens'].tolist(), data['labels'].tolist(), test_size=0.2, random_state=42)

# Flatten the list of tokens for each example
X_train_flat = [token for sublist in X_train for token in sublist]
X_test_flat = [token for sublist in X_test for token in sublist]

# Flatten the list of labels for each example
y_train_flat = [label for sublist in y_train for label in sublist]
y_test_flat = [label for sublist in y_test for label in sublist]


In [71]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train_flat)
X_test_vectorized = vectorizer.transform(X_test_flat)

# Create and train the MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(128, 128), random_state=1)
clf.fit(X_train_vectorized, y_train_flat)

# Predict on the test set
y_pred = clf.predict(X_test_vectorized)

# Evaluate the performance
print(classification_report(y_test_flat, y_pred))



KeyboardInterrupt



In [ ]:
df.head()